In [2]:
!ls -alr

total 308
drwxr-xr-x 14 root root     6144 Aug 14 12:47 wandb
drwxr-xr-x  6 root root     6144 Aug 14 12:47 trained_model
drwxr-xr-x  2 root root     6144 Aug 12 14:05 trained-model
drwxr-xr-x  2 root root     6144 Aug 14 08:39 tmp_trainer
-rw-r--r--  1 root root    25087 Aug 12 14:09 tag.le
drwxr-xr-x 13 root root     6144 Aug 14 14:42 results-v3-large
drwxr-xr-x  8 root root     6144 Aug 13 15:14 results-v2-large
drwxr-xr-x 22 root root     6144 Aug 12 19:00 results-mini
drwxr-xr-x 36 root root     6144 Aug 12 09:45 results
drwxr-xr-x  2 root root     6144 Aug  9 13:10 model
drwxr-xr-x  6 root root     6144 Aug 14 09:34 data
-rw-r--r--  1 root root    53781 Aug 14 14:48 TagClassification.ipynb
-rw-r--r--  1 root root    56805 Aug 14 14:47 TagClassification-Mini.ipynb
-rw-r--r--  1 root root    35071 Aug 14 12:43 DataPreparation.ipynb
-rw-r--r--  1 root root    38630 Aug 13 14:12 DataPreparation-Mini.ipynb
-rw-r--r--  1 root root      211 Aug 12 19:51 .yarnrc
-rw-r--r--  1 root root  

In [5]:
!pip install -q transformers[torch] accelerate
!pip3 install -q wandb


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import json 
import pandas as pd 
import os 
import pickle

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np 

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.preprocessing import LabelEncoder


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# !pip3 uninstall -q wandb -y

In [1]:
import wandb

In [3]:
wandb.login(key="4de6103347df6561e7258cdef0ef60bbc1233695", relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
MODEL_DIRECTORY = "./trained_model/v3-large"

In [8]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_attention_mask=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [9]:
train_df = pd.read_json("./data/dataset/v4/train.json")

In [10]:
print(train_df.shape)

(9705, 7)


In [11]:
texts = train_df['section_content'].values
labels_text = train_df['tags'].values

In [12]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels_text)

In [14]:
os.makedirs(MODEL_DIRECTORY)

In [15]:
with open(os.path.join(MODEL_DIRECTORY, "tag.le"), 'wb') as file: 
    pickle.dump(label_encoder, file)

In [16]:
num_labels = train_df.tags.unique().shape[0]
print("Number of labels", num_labels)

Number of labels 24


In [ ]:
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Load Model

In [17]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load Dataset

In [18]:
dataset = TextDataset(texts, labels, tokenizer, max_length=512)

## Compute Metrics callback

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_score(labels, predictions, )
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [21]:

training_args = TrainingArguments(
    output_dir='./results-v3-large',
    run_name="tag-classifier-v3-large",
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    logging_dir='./logs-v3-large',
    logging_steps=10,
    learning_rate=2e-5,
    eval_strategy="epoch"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset, 
    compute_metrics=compute_metrics

)

# Train the model
trainer.train()


wandb: Currently logged in as: anurag-credcore (credcore). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.404100,0.550005,0.862854,0.083708,0.099677,0.080602
2,0.282500,0.257680,0.935600,0.562615,0.563232,0.537731
3,0.240800,0.160999,0.954560,0.696816,0.798513,0.730761
4,0.147600,0.114396,0.967543,0.786666,0.844065,0.808718
5,0.136800,0.094020,0.971767,0.825907,0.925909,0.871864
6,0.085000,0.074743,0.974652,0.857018,0.927558,0.886557
7,0.063400,0.055640,0.982380,0.927130,0.925240,0.917216
8,0.030700,0.043230,0.985781,0.881064,0.950685,0.913416
9,0.048000,0.028265,0.991448,0.949222,0.984031,0.964452
10,0.033200,0.025446,0.992169,0.953418,0.989024,0.970103


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [23]:
# Save the model and tokenizer to disk
model.save_pretrained(MODEL_DIRECTORY)
tokenizer.save_pretrained(MODEL_DIRECTORY)

('./trained_model/v3-large/tokenizer_config.json',
 './trained_model/v3-large/special_tokens_map.json',
 './trained_model/v3-large/vocab.txt',
 './trained_model/v3-large/added_tokens.json')

# Load Model For Inferences

In [6]:
import json 
import pandas as pd 
import os 
import pickle

In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np 

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.preprocessing import LabelEncoder


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model = DistilBertForSequenceClassification.from_pretrained(MODEL_DIRECTORY)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_DIRECTORY)

In [12]:
test_df = pd.read_json("./data/dataset/v4/test.json")
test_df = test_df.drop_duplicates("section_content")


In [14]:
with open(os.path.join(MODEL_DIRECTORY, "tag.le"), 'rb') as file: 
    label_encoder = pickle.load(file)

In [15]:
test_texts = test_df['section_content'].values
test_labels = test_df['tags'].values
test_labels = label_encoder.transform(test_labels)

In [16]:
from sklearn.metrics import classification_report

In [17]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_attention_mask=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [18]:
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_length=512)

In [19]:
inference_args = TrainingArguments(
    output_dir='./results',  
    per_device_eval_batch_size=32, 
    dataloader_drop_last=False,  
    no_cuda=False if torch.cuda.is_available() else True, 
    seed=42,  
    report_to="none" # Disable wandb reporting. 
)
trainer = Trainer(
    model=model, 
    args = inference_args
)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [20]:
# Make predictions on the test set
predictions = trainer.predict(test_dataset)

# Extract the logits and convert to predicted labels
logits = predictions.predictions
predicted_labels = np.argmax(logits, axis=-1)

# Convert numeric labels back to original text labels for comparison
predicted_labels_text = label_encoder.inverse_transform(predicted_labels)
true_labels_text = label_encoder.inverse_transform(test_labels)

# Generate a classification report
report = classification_report(true_labels_text, predicted_labels_text, target_names=label_encoder.classes_)


In [21]:
print(report)

                                    precision    recall  f1-score   support

                  Additional Liens       0.74      0.85      0.79        20
                 Asset Disposition       0.67      0.77      0.71        13
            Compliance Certificate       0.50      0.67      0.57         6
           Consequences of Default       0.69      0.71      0.70        31
                  Event of Default       0.67      1.00      0.80         6
                 Events of Default       0.83      0.59      0.69        17
           Facilities / Instrument       0.76      0.83      0.79        35
                Financial Covenant       0.94      0.80      0.86        20
              Financial Statements       0.67      0.80      0.73         5
                    Governing Laws       0.90      0.95      0.92        19
            Incremental Facilities       0.67      0.50      0.57         8
                     Interest Rate       0.82      0.86      0.84        21
           

In [ ]:
all_df = pd.read_json("./data/all-data.json")
print(all_df.shape)
all_df = all_df[all_df.word_count > 20]
print(all_df.shape)

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_df['predicted_tags'] = predicted_labels_text

In [ ]:
test_df.head()

In [ ]:
group_df = all_df[['section_content', 'tags']].groupby('section_content').agg(list).reset_index()
group_df['n_tags'] = group_df.tags.apply(lambda x: len(x))
group_df['n_unique_tags'] = group_df.tags.apply(lambda x: len(set(x)))
print(group_df[group_df.n_tags > 1].shape)
print(group_df[group_df.n_unique_tags > 1].shape)
group_df.sort_values(by='n_unique_tags', ascending=False).head(10)

In [ ]:
test_df[["section_content", "predicted_tags"]].merge(all_df.drop_duplicates("section_content"), how="left", on="section_content", validate="one_to_one").to_excel("./data/output/test_data_predictions.xlsx")

In [ ]:
test_df.shape

In [ ]:
print(report)

In [ ]:
def predict_labels(texts, model, tokenizer, label_encoder, max_length=512, device='cuda'):
    # Move model to the specified device
    model.to(device)
    
    # Tokenize the texts
    encodings = tokenizer(
        texts,
        max_length=max_length,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    
    # Move tensors to the same device as the model
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    
    # Put the model in evaluation mode
    model.eval()
    
    # Perform the inference
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    
    # Move predictions back to the CPU before converting to numpy
    predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())
    
    return predicted_labels

In [ ]:
predicted_labels_text = predict_labels(test_texts.tolist(), model, tokenizer, label_encoder)
true_labels_text = label_encoder.inverse_transform(test_labels)

In [ ]:
!nvidia-smi

In [ ]:
# Extract the logits and convert to predicted labels
# logits = predictions.predictions
# predicted_labels = np.argmax(logits, axis=-1)

# Convert numeric labels back to original text labels for comparison
# Generate a classification report
report = classification_report(true_labels_text, predicted_labels_text, target_names=label_encoder.classes_)


In [ ]:
print(report)